In [1]:
import json
import pandas as pd
from isaac.constants import BASIC_TRAINING_COLS

In [2]:
hdf_path = "../new_exp_data/exp7_passive.h5"
trial_i = 0

for condition_id in range(1, 5):
    filename = "../new_exp_data/physics_data%d.json" % condition_id
    fd = open(filename)
    sim_data = json.load(fd)
    
    
    for sim in sim_data:
        trial = pd.DataFrame(columns=sim.keys())
        
        for key in sim.keys():
            if key in ["tR", "tM" , "practice", "refreshes", "trial_type", "condition_world_variant", "world_id"]:
                trial[key] = [sim[key]] * 2700
            else:
                trial[key] = sim[key]
                
        trial["A"] = (trial.tM == "A")
        trial["B"] = (trial.tM == "B")
        trial["same"] = (trial.tM == "same")
        
        trial["attract"] = (trial.tR == 3)
        trial["none"] = (trial.tR == 0)
        trial["repel"] = (trial.tR == -3)
        
        trial["condition"] = condition_id
                
        if trial["practice"].unique()[0]:
            continue 
            
        trial.to_hdf(path_or_buf=hdf_path, key="trial_"+str(trial_i))
        trial_i += 1

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['refreshes', 'tM']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [3]:
!ls -lah ../new_exp_data/

total 265M
drwxr-xr-x  4 hector hector 4.0K Jan 13 14:21 .
drwxr-xr-x 10 hector hector  36K Dec 26 11:24 ..
-rw-rw-r--  1 hector hector  37K Jan 13 14:20 e7_passive_io_rtheta.rdata
-rw-rw-r--  1 hector hector  20K Dec 26 16:32 e7_passive.rdata
-rw-r--r--  1 hector hector 227M Jan 18 15:07 exp7_passive.h5
-rw-rw-r--  1 hector hector 7.4M Dec 26 11:22 Experiment_7_passive.zip
drwxr-xr-x  2 hector hector 4.0K Dec 26 17:31 .ipynb_checkpoints
drwxr-xr-x  4 hector hector 4.0K Jan 13 14:21 pd_summaries
-rw-rw-r--  1 hector hector  17M Jan 13 14:20 pd_summaries.zip
-rw-rw-r--  1 hector hector 3.7M Dec 26 16:30 physics_data1.json
-rw-rw-r--  1 hector hector 3.7M Dec 26 16:30 physics_data2.json
-rw-rw-r--  1 hector hector 3.7M Dec 26 16:30 physics_data3.json
-rw-rw-r--  1 hector hector 3.7M Dec 26 16:30 physics_data4.json
-rw-rw-r--  1 hector hector 1020 Dec 26 11:22 world_key.rdata


# Load participant answers

In [4]:
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()

In [5]:
rdata_path = "../new_exp_data/e7_passive.rdata"
r['load'](rdata_path)

'sw','tw','feedback'


In [6]:
trials = r["tw"]
trials = trials.query("practice == 0")

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [7]:
trials.head()

,ppt,trial_type,world_id,practice,cond_worldvar,true_rel,true_mass,resp_rel,resp_mass,conf_rel,conf_mass,cor_rel,cor_mass,trial_type.f
2,1,1,148,0,4,attract,same,attract,A,100.0,97.0,1,0,att.same
3,1,7,597,0,4,repel,same,attract,same,87.0,77.0,0,1,rep.same
4,1,2,1282,0,4,attract,A,attract,B,99.0,97.0,1,0,att.A
5,1,9,2022,0,4,repel,B,repel,B,99.0,98.0,1,1,rep.B
6,1,8,1251,0,4,repel,A,repel,A,95.0,84.0,1,1,rep.A


In [8]:
count = 0
for name, df in trials.groupby(["cond_worldvar", "world_id"]):
    count += 1
    
    print(df.resp_rel.mode())
    for response in ["'attract'", "'repel'", "'none'"]:
        print(response, df.query("resp_rel == "+response).shape[0])
    
    print(df.resp_mass.mode())
    print("->", df.true_mass.unique())
    for response in ["'A'", "'B'", "'same'"]:
        print(response, df.query("resp_mass == "+response).shape[0])
    
    print()

0    repel
dtype: object
'attract' 4
'repel' 19
'none' 2
0    A
dtype: object
-> ['A']
'A' 13
'B' 5
'same' 7

0    repel
dtype: object
'attract' 2
'repel' 13
'none' 10
0    B
dtype: object
-> ['B']
'A' 4
'B' 13
'same' 8

0    repel
dtype: object
'attract' 8
'repel' 11
'none' 6
0    B
dtype: object
-> ['B']
'A' 1
'B' 18
'same' 6

0    attract
dtype: object
'attract' 16
'repel' 8
'none' 1
0    B
dtype: object
-> ['B']
'A' 5
'B' 15
'same' 5

0    repel
dtype: object
'attract' 4
'repel' 14
'none' 7
0       B
1    same
dtype: object
-> ['same']
'A' 7
'B' 9
'same' 9

0    attract
dtype: object
'attract' 12
'repel' 10
'none' 3
0    A
dtype: object
-> ['same']
'A' 13
'B' 2
'same' 10

0    none
dtype: object
'attract' 5
'repel' 9
'none' 11
0    A
1    B
dtype: object
-> ['same']
'A' 9
'B' 9
'same' 7

0    repel
dtype: object
'attract' 6
'repel' 11
'none' 8
0    A
dtype: object
-> ['A']
'A' 16
'B' 4
'same' 5

0    attract
dtype: object
'attract' 12
'repel' 10
'none' 3
0    same
dtype: object
-> 